**This is a subheading**

In [ ]:
!pip install requests
!pip install django

import requests
import os
from django.conf import settings

def get_weather_data(lat, lon):
    api_key = os.getenv('OPENWEATHER_API_KEY')  # Store in .env
    url = 'https://api.openweathermap.org/data/3.0/onecall'
    params = {
        'lat': 36.600238,
        'lon': -121.894676,
        'appid': '320b363a0425ce842cc0235911598a51',
        'units': 'metric',
        'lang': 'eng',
        'exclude': 'minutely,alerts'
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise error for bad status
        return response.json()  # Returns current, hourly, daily data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
        return None



In [ ]:
!pip install rest_framework.decorators
!pip install rest_framework.response



ERROR: Could not find a version that satisfies the requirement rest_framework.decorators (from versions: none)
ERROR: No matching distribution found for rest_framework.decorators
ERROR: Could not find a version that satisfies the requirement rest_framework.response (from versions: none)
ERROR: No matching distribution found for rest_framework.response


ModuleNotFoundError: No module named 'rest_framework'

In [ ]:
!pip install djangorestframework
# Example usage in Django view
from djangorestframework import api_view
from djangorestframework import Response

@api_view(['GET'])
def weather_data(request, farm_id):
    # Fetch farm coordinates from database
    # farm = Farm.objects.get(id=farm_id)  # Assume Farm model with lat/lon
    # weather = get_weather_data(farm.latitude, farm.longitude)
    # if weather:
    #     return Response(weather)
    # return Response({"error": "Failed to fetch weather data"}, status=500)
    # Example usage with hardcoded coordinates
    weather = get_weather_data(36.600238, -121.894676)
    if weather:
        return Response(weather)
    return Response({"error": "Failed to fetch weather data"}, status=500)

ModuleNotFoundError: No module named 'djangorestframework'

In [ ]:
import os
import requests
from datetime import datetime

# 1) Define a simple OpenWeather adapter
class OpenWeatherProvider:
    BASE_URL = 'https://api.openweathermap.org/data/3.0/onecall'

    def __init__(self, api_key: str):
        self.api_key = '13453137d1b56511802304e7024fb57f'

    def fetch(self, lat: float, lon: float) -> dict:
        params = {
            'lat': lat,
            'lon': lon,
            'appid': self.api_key,
            'units': 'metric',
            'exclude': 'minutely,alerts'
        }
        resp = requests.get(self.BASE_URL, params=params)
        resp.raise_for_status()
        return resp.json()

# 2) Monterey’s coords
monterey_lat = 36.600238
monterey_lon = -121.894676

# 3) Instantiate & fetch
ow = OpenWeatherProvider(api_key=os.getenv('OPENWEATHER_API_KEY'))
weather_data = ow.fetch(monterey_lat, monterey_lon)

# 4) Example: print today’s current temp & next 3‑day forecast
current_temp = weather_data['current']['temp']
print(f"As of {datetime.fromtimestamp(weather_data['current']['dt'])}: {current_temp}°C in Monterey")

for day in weather_data['daily'][:3]:
    date = datetime.fromtimestamp(day['dt']).date()
    print(f"  • {date}: {day['weather'][0]['description'].title()}, "
          f"{day['temp']['min']}°C ↔ {day['temp']['max']}°C")


As of 2025-07-16 22:38:38: 22.02°C in Monterey
  • 2025-07-16: Broken Clouds, 13.83°C ↔ 22.71°C
  • 2025-07-17: Clear Sky, 11.54°C ↔ 18.06°C
  • 2025-07-18: Clear Sky, 12.94°C ↔ 17.19°C


In [3]:
import requests
print(requests.get("https://httpbin.org/ip").json())


{'origin': '34.143.138.197'}


In [2]:
BASE_URL = "https://api.openweathermap.org/data/3.0/onecall"


In [4]:
r = requests.get(
    "https://api.openweathermap.org/data/3.0/onecall",
    params={"lat":0, "lon":0, "appid": '13453137d1b56511802304e7024fb57f'}
)
print(r.status_code, r.text[:200])


200 {"lat":0,"lon":0,"timezone":"Etc/GMT","timezone_offset":0,"current":{"dt":1752874762,"sunrise":1752818548,"sunset":1752862175,"temp":297.35,"feels_like":297.55,"pressure":1018,"humidity":66,"dew_point


In [6]:
BASE_URL = "https://api.openweathermap.org/data/2.5/onecall"

In [9]:
!pip install ace_tools

In [13]:
#Fetch Global Weather & Train Irrigation Model

# 1) Install dependencies (run once)
!pip install requests pandas scikit-learn ace_tools

import os
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# 2) Set your OpenWeatherMap API key
OWM_API_KEY = '13453137d1b56511802304e7024fb57f'
os.environ['OPENWEATHER_API_KEY'] = OWM_API_KEY

# 3) OpenWeather provider adapter
class OpenWeatherProvider:
    BASE_URL = 'https://api.openweathermap.org/data/3.0/onecall'
    def __init__(self, api_key: str):
        self.api_key = '13453137d1b56511802304e7024fb57f'
    def fetch(self, lat: float, lon: float) -> dict:
        params = {
            'lat': lat,
            'lon': lon,
            'appid': self.api_key,
            'units': 'metric',
            'exclude': 'minutely,alerts'
        }
        r = requests.get(self.BASE_URL, params=params)
        r.raise_for_status()
        return r.json()

# 4) Helper to derive season (Northern Hemisphere)
def season_from_date(date):
    m = date.month
    if m in (3,4,5):   return 'spring'
    if m in (6,7,8):   return 'summer'
    if m in (9,10,11): return 'autumn'
    return 'winter'

# 5) Define your farms with metadata
farms = [
    {'id':1, 'lat':36.600238,  'lon':-121.894676, 'soil_type':'sandy', 'crop_type':'lettuce'},
    {'id':2, 'lat':-1.2920669, 'lon':36.8219462,  'soil_type':'loamy', 'crop_type':'maize'},
    {'id':3, 'lat':-23.5505199,'lon':-46.6333094, 'soil_type':'clay',  'crop_type':'soy'},
    {'id':4, 'lat':13.756331,  'lon':100.501765,  'soil_type':'silt',  'crop_type':'rice'},
]

# 6) Fetch weather + build DataFrame
provider = OpenWeatherProvider(api_key=OWM_API_KEY)
records = []
now = datetime.now()

for farm in farms:
    data = provider.fetch(farm['lat'], farm['lon'])
    temp  = data['current']['temp']
    wind  = data['current']['wind_speed']
    rain  = data.get('daily',[{}])[0].get('rain', 0.0)
    records.append({
        'farm_id':     farm['id'],
        'latitude':    farm['lat'],
        'longitude':   farm['lon'],
        'soil_type':   farm['soil_type'],
        'crop_type':   farm['crop_type'],
        'temperature': temp,
        'rainfall':    rain,
        'wind_speed':  wind,
        'season':      season_from_date(now)
    })

df = pd.DataFrame(records)

# 7) One‑hot encode categorical features
df_encoded = pd.get_dummies(df,
                            columns=['soil_type','crop_type','season'],
                            drop_first=True)

# 8) Create synthetic irrigation_need target (replace with real labels)
np.random.seed(42)
df_encoded['irrigation_need'] = (
    100
    - df_encoded['rainfall'] * 3
    + (df_encoded['temperature'] - 20) * 2
    + df_encoded['wind_speed'] * 1.5
    + np.random.normal(0, 5, size=len(df_encoded))
)

# 9) Prepare features & split
features = [c for c in df_encoded.columns if c not in ['farm_id','irrigation_need']]
X = df_encoded[features]
y = df_encoded['irrigation_need']
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    random_state=42)

# 10) Train Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 11) Evaluate performance
preds = model.predict(X_test)
print(f"MAE: {mean_absolute_error(y_test, preds):.2f}")
print(f"R²:  {r2_score(y_test, preds):.2f}")

# 12) Show feature importances
importances = pd.Series(model.feature_importances_, index=features)\
                .sort_values(ascending=False)
print("\nFeature Importances:\n", importances)


MAE: 21.94
R²:  nan

Feature Importances:
 temperature        0.190017
wind_speed         0.184671
latitude           0.175761
longitude          0.107664
soil_type_silt     0.094118
soil_type_sandy    0.079499
crop_type_rice     0.060606
crop_type_soy      0.055259
rainfall           0.052405
soil_type_loamy    0.000000
crop_type_maize    0.000000
dtype: float64


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [11]:
# 1) Install requests (if you haven’t already)
!pip install requests

# 2) Test connectivity & API key
import requests

OWM_API_KEY = '13453137d1b56511802304e7024fb57f'  # ← use your real key here

r = requests.get(
    "https://api.openweathermap.org/data/3.0/onecall",
    params={"lat": 0, "lon": 0, "appid": OWM_API_KEY}
)
print("Status code:", r.status_code)
print("Body snippet:", r.text[:200])


Status code: 200
Body snippet: {"lat":0,"lon":0,"timezone":"Etc/GMT","timezone_offset":0,"current":{"dt":1752875230,"sunrise":1752818548,"sunset":1752862175,"temp":297.35,"feels_like":297.55,"pressure":1018,"humidity":66,"dew_point


In [14]:
# ──────────────────────────────────────────────────────────────────────────────
# Colab Cell: Fetch Global Weather & Train Irrigation Model (no ace_tools)
# ──────────────────────────────────────────────────────────────────────────────

# 1) Install dependencies
!pip install requests pandas scikit-learn

import os
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from IPython.display import display

# 2) Set your OpenWeatherMap API key
OWM_API_KEY = '13453137d1b56511802304e7024fb57f'  # ← Replace with your key
os.environ['OPENWEATHER_API_KEY'] = OWM_API_KEY

# 3) OpenWeather provider adapter
class OpenWeatherProvider:
    BASE_URL = 'https://api.openweathermap.org/data/3.0/onecall'
    def __init__(self, api_key: str):
        self.api_key = api_key
    def fetch(self, lat: float, lon: float) -> dict:
        params = {
            'lat': lat, 'lon': lon,
            'appid': self.api_key,
            'units': 'metric',
            'exclude': 'minutely,alerts'
        }
        r = requests.get(self.BASE_URL, params=params)
        r.raise_for_status()
        return r.json()

# 4) Season helper (Northern Hemisphere)
def season_from_date(date):
    m = date.month
    if m in (3,4,5):   return 'spring'
    if m in (6,7,8):   return 'summer'
    if m in (9,10,11): return 'autumn'
    return 'winter'

# 5) Farms metadata
farms = [
    {'id':1, 'lat':36.600238,  'lon':-121.894676, 'soil_type':'sandy', 'crop_type':'lettuce'},
    {'id':2, 'lat':-1.2920669, 'lon':36.8219462,  'soil_type':'loamy', 'crop_type':'maize'},
    {'id':3, 'lat':-23.5505199,'lon':-46.6333094, 'soil_type':'clay',  'crop_type':'soy'},
    {'id':4, 'lat':13.756331,  'lon':100.501765,  'soil_type':'silt',  'crop_type':'rice'},
]

# 6) Fetch weather + build DataFrame
prov = OpenWeatherProvider(api_key=OWM_API_KEY)
rows = []
now = datetime.now()
for farm in farms:
    d = prov.fetch(farm['lat'], farm['lon'])
    temp = d['current']['temp']
    wind = d['current']['wind_speed']
    rain = d.get('daily',[{}])[0].get('rain',0.0)
    rows.append({
        'farm_id':   farm['id'],
        'latitude':  farm['lat'],
        'longitude': farm['lon'],
        'soil_type': farm['soil_type'],
        'crop_type': farm['crop_type'],
        'temperature': temp,
        'rainfall':    rain,
        'wind_speed':  wind,
        'season':      season_from_date(now)
    })
df = pd.DataFrame(rows)

# 7) One‑hot encode categorical features
df_encoded = pd.get_dummies(df,
                            columns=['soil_type','crop_type','season'],
                            drop_first=True)

# 8) Synthetic target (replace when you have real labels)
np.random.seed(42)
df_encoded['irrigation_need'] = (
    100
    - df_encoded['rainfall'] * 3
    + (df_encoded['temperature'] - 20) * 2
    + df_encoded['wind_speed'] * 1.5
    + np.random.normal(0, 5, size=len(df_encoded))
)

# 9) Prepare features & split
feat_cols = [c for c in df_encoded.columns if c not in ['farm_id','irrigation_need']]
X, y = df_encoded[feat_cols], df_encoded['irrigation_need']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# 10) Train & evaluate model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print(f"MAE: {mean_absolute_error(y_test, preds):.2f}")
print(f"R²:  {r2_score(y_test, preds):.2f}")

# 11) Feature importances
importances = pd.Series(model.feature_importances_, index=feat_cols).sort_values(ascending=False)
print("\nFeature Importances:\n", importances)

# 12) Inspect the final dataset
display(df_encoded)


MAE: 21.94
R²:  nan

Feature Importances:
 temperature        0.190017
wind_speed         0.184671
latitude           0.175761
longitude          0.107664
soil_type_silt     0.094118
soil_type_sandy    0.079499
crop_type_rice     0.060606
crop_type_soy      0.055259
rainfall           0.052405
soil_type_loamy    0.000000
crop_type_maize    0.000000
dtype: float64


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


,farm_id,latitude,longitude,temperature,rainfall,wind_speed,soil_type_loamy,soil_type_sandy,soil_type_silt,crop_type_maize,crop_type_rice,crop_type_soy,irrigation_need
0,1,36.600238,-121.894676,23.98,0.00,3.89,False,True,False,False,False,False,116.278571
1,2,-1.292067,36.821946,16.21,0.22,1.05,True,False,False,True,False,False,92.643678
2,3,-23.550520,-46.633309,14.66,0.00,3.91,False,False,False,False,False,True,98.423443
3,4,13.756331,100.501765,29.17,0.77,2.77,False,False,True,False,True,False,127.800149


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['farm_id'].plot(kind='hist', bins=20, title='farm_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_25['soil_type_sandy'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_25, x='index', y='soil_type_sandy', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_26['soil_type_silt'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_26, x='index', y='soil_type_silt', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_27['crop_type_maize'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_27, x='index', y='crop_type_maize', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [16]:
# Synthetic Global Farms + Weather → Irrigation Model

# 1) Install dependencies (run once)
!pip install requests pandas scikit-learn

import os
import time
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from IPython.display import display

# 2) Set your OpenWeatherMap API key
OWM_API_KEY = '13453137d1b56511802304e7024fb57f'
os.environ['OPENWEATHER_API_KEY'] = OWM_API_KEY

# 3) Define the OpenWeather provider adapter
class OpenWeatherProvider:
    BASE_URL = 'https://api.openweathermap.org/data/3.0/onecall'
    def __init__(self, api_key: str):
        self.api_key = '13453137d1b56511802304e7024fb57f'
    def fetch(self, lat: float, lon: float) -> dict:
        params = {'lat': lat, 'lon': lon, 'appid': self.api_key,
                  'units': 'metric', 'exclude': 'minutely,alerts'}
        resp = requests.get(self.BASE_URL, params=params)
        resp.raise_for_status()
        return resp.json()

# 4) Season helper
def season_from_date(date):
    m = date.month
    if m in (3,4,5):   return 'spring'
    if m in (6,7,8):   return 'summer'
    if m in (9,10,11): return 'autumn'
    return 'winter'

# 5) Generate synthetic farm metadata (e.g., 20 farms worldwide)
np.random.seed(0)
num_farms = 20
lats = np.random.uniform(-60, 60, num_farms)
lons = np.random.uniform(-180, 180, num_farms)
soil_types = ['sandy', 'loamy', 'clay', 'silt']
crop_types = ['rice', 'maize', 'soy', 'wheat', 'lettuce']

farms = []
for i in range(num_farms):
    farms.append({
        'id': i+1,
        'lat': float(lats[i]),
        'lon': float(lons[i]),
        'soil_type': np.random.choice(soil_types),
        'crop_type': np.random.choice(crop_types)
    })

# 6) Fetch weather + build DataFrame
provider = OpenWeatherProvider(api_key=OWM_API_KEY)
records = []
now = datetime.now()

for farm in farms:
    data = provider.fetch(farm['lat'], farm['lon'])
    temp = data['current']['temp']
    wind = data['current']['wind_speed']
    rain = data.get('daily', [{}])[0].get('rain', 0.0)

    records.append({
        'farm_id':     farm['id'],
        'latitude':    farm['lat'],
        'longitude':   farm['lon'],
        'soil_type':   farm['soil_type'],
        'crop_type':   farm['crop_type'],
        'temperature': temp,
        'rainfall':    rain,
        'wind_speed':  wind,
        'season':      season_from_date(now)
    })
    # avoid rate-limits
    time.sleep(1)

df = pd.DataFrame(records)

# 7) One‑hot encode categorical features
df_encoded = pd.get_dummies(df,
                            columns=['soil_type','crop_type','season'],
                            drop_first=True)

# 8) Synthetic target (replace with your real irrigation data)
np.random.seed(42)
df_encoded['irrigation_need'] = (
    100
    - df_encoded['rainfall'] * 3
    + (df_encoded['temperature'] - 20) * 2
    + df_encoded['wind_speed'] * 1.5
    + np.random.normal(0, 5, size=len(df_encoded))
)

# 9) Prepare features & split
feat_cols = [c for c in df_encoded.columns if c not in ['farm_id','irrigation_need']]
X, y = df_encoded[feat_cols], df_encoded['irrigation_need']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# 10) Train & evaluate model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
preds = model.predict(X_test)

print(f"MAE: {mean_absolute_error(y_test, preds):.2f}")
print(f"R²:  {r2_score(y_test, preds):.2f}\n")

# 11) Feature importances
importances = pd.Series(model.feature_importances_, index=feat_cols)\
                .sort_values(ascending=False)
print("Feature Importances:\n", importances)

# 12) Inspect the final dataset
display(df_encoded)


MAE: 4.90
R²:  -0.01

Feature Importances:
 rainfall           0.398422
temperature        0.378617
wind_speed         0.118704
latitude           0.036977
longitude          0.036474
soil_type_silt     0.011475
crop_type_rice     0.005669
crop_type_soy      0.004645
soil_type_loamy    0.003972
crop_type_maize    0.002152
soil_type_sandy    0.001449
crop_type_wheat    0.001444
dtype: float64


,farm_id,latitude,longitude,temperature,rainfall,wind_speed,soil_type_loamy,soil_type_sandy,soil_type_silt,crop_type_maize,crop_type_rice,crop_type_soy,crop_type_wheat,irrigation_need
0,1,5.857620,172.302603,28.32,11.35,7.56,False,False,False,False,True,False,False,96.413571
1,2,25.822724,107.697083,20.59,3.73,1.18,True,False,False,True,False,False,False,91.068678
2,3,12.331605,-13.867430,23.99,3.23,2.42,False,False,True,True,False,False,False,105.158443
3,4,5.385982,100.990503,20.96,0.00,0.88,True,False,False,False,False,False,True,110.855149
4,5,-9.161424,-137.421207,25.83,0.33,8.02,False,False,False,False,False,False,True,121.529233
5,6,17.507294,50.371568,30.80,0.00,4.31,False,False,False,False,False,True,False,126.894315
6,7,-7.489535,-128.392817,25.40,0.61,10.25,False,False,True,False,True,False,False,132.241064
7,8,47.012760,160.080810,9.77,3.84,3.18,False,False,False,False,False,False,True,76.627174
8,9,55.639531,7.865396,18.28,0.00,0.99,True,False,False,False,False,False,False,95.697628
9,10,-13.987018,-30.721702,24.71,0.00,7.26,True,False,False,False,False,True,False,123.022800


from matplotlib import pyplot as plt
_df_28['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_29['farm_id'].plot(kind='hist', bins=20, title='farm_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_30['latitude'].plot(kind='hist', bins=20, title='latitude')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_53['soil_type_sandy'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_53, x='index', y='soil_type_sandy', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_54['soil_type_silt'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_54, x='index', y='soil_type_silt', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_55['crop_type_maize'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_55, x='index', y='crop_type_maize', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [1]:
# Modularized & Scalable AGRO‑AI Data Pipeline Prototype

# 0) Install dependencies (run once)
!pip install requests pandas scikit-learn tqdm tenacity joblib

import os
import time
import requests
import logging
import joblib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from functools import lru_cache
from tenacity import retry, stop_after_attempt, wait_exponential
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, r2_score
from tqdm.auto import tqdm

# 1) Configuration & Parameters
# Farm metadata source (CSV on Drive or local)
FARM_CSV = 'farms_metadata.csv'  # path to CSV with columns: id,lat,lon,soil_type,crop_type
# Date range for historical weather pulls (e.g. last N days)
DAYS_OF_HISTORY = 30
# Cross-val folds & hyperparameter search
CV_FOLDS = 5
HYPER_SEARCH_ITERS = 20
# Output paths
MODEL_PATH = 'rf_irrigation_model.joblib'
DATA_PATH = 'processed_irrigation_data.csv'

# 2) Setup logging
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO)
logger = logging.getLogger('agro_ai')

# ──────────────────────────────────────────────────────────────────────────────
# 3) Load farm metadata
# ──────────────────────────────────────────────────────────────────────────────
farms_df = pd.read_csv(FARM_CSV)

# ──────────────────────────────────────────────────────────────────────────────
# 4) Weather API adapter with caching & retry
# ──────────────────────────────────────────────────────────────────────────────
@lru_cache(maxsize=None)
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, max=10))
def fetch_weather_day(lat, lon, date_str: str) -> dict:
    """Fetch historical daily data via One Call API (timemachine endpoint)."""
    url = 'https://api.openweathermap.org/data/3.0/onecall/timemachine'
    params = {
        'lat': lat,
        'lon': lon,
        'dt': int(datetime.fromisoformat(date_str).timestamp()),
        'appid': os.getenv('13453137d1b56511802304e7024fb57f'),
        'units': 'metric'
    }
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    return resp.json()

# ──────────────────────────────────────────────────────────────────────────────
# 5) Feature extraction for one farm on one date
# ──────────────────────────────────────────────────────────────────────────────

def extract_features_for_date(farm: pd.Series, date: datetime) -> dict:
    lat, lon = farm['lat'], farm['lon']
    date_str = date.strftime('%Y-%m-%dT%H:%M:%S')
    data = fetch_weather_day(lat, lon, date_str)
    current = data.get('current', {})
    features = {
        'farm_id': farm['id'],
        'date': date.date(),
        'latitude': lat,
        'longitude': lon,
        'temperature': current.get('temp'),
        'rainfall': sum(h.get('rain', 0) for h in data.get('hourly', [])),
        'wind_speed': current.get('wind_speed'),
        'humidity': current.get('humidity'),
        'pressure': current.get('pressure'),
        'dew_point': current.get('dew_point'),
        'clouds': current.get('clouds'),
        'uv_index': current.get('uvi'),
    }
    # Add static metadata
    features['soil_type'] = farm['soil_type']
    features['crop_type'] = farm['crop_type']
    # Season
    m = date.month
    features['season'] = ('spring' if m in (3,4,5) else
                          'summer' if m in (6,7,8) else
                          'autumn' if m in (9,10,11) else
                          'winter')
    return features

# ──────────────────────────────────────────────────────────────────────────────
# 6) Build dataset across farms & dates
# ──────────────────────────────────────────────────────────────────────────────
records = []
end_date = datetime.utcnow()
for farm in tqdm(farms_df.to_dict(orient='records'), desc='Farms'):
    for days_back in tqdm(range(DAYS_OF_HISTORY), leave=False, desc='Dates'):
        day = end_date - timedelta(days=days_back)
        try:
            rec = extract_features_for_date(pd.Series(farm), day)
            records.append(rec)
        except Exception as e:
            logger.warning(f"Failed {farm['id']} @ {day.date()}: {e}")
        time.sleep(1)  # rate-limit

df = pd.DataFrame(records)

# 7) Encode categoricals & save
categorical_cols = ['soil_type','crop_type','season']
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
df_encoded.to_csv(DATA_PATH, index=False)
logger.info(f"Saved processed data to {DATA_PATH}")

# 8) Prepare features & label
X = df_encoded.drop(['farm_id','date','irrigation_need'], axis=1, errors='ignore')
# TODO: Replace synthetic irrigation_need with real sensor logs once available
np.random.seed(0)
df_encoded['irrigation_need'] = (
    100 - df_encoded['rainfall']*3 + (df_encoded['temperature']-20)*2
    + df_encoded['wind_speed']*1.5 + np.random.normal(0,5, len(df_encoded))
)
y = df_encoded['irrigation_need']

# 9) Cross-validation
cv_scores = cross_val_score(RandomForestRegressor(), X, y, cv=CV_FOLDS, scoring='neg_mean_absolute_error')
logger.info(f"CV MAE: {-cv_scores.mean():.2f} ± {cv_scores.std():.2f}")

# 10) Hyperparameter tuning
param_dist = {
    'n_estimators': [50,100,200],
    'max_depth': [None,10,20],
    'min_samples_leaf': [1,2,4]
}
search = RandomizedSearchCV(
    RandomForestRegressor(random_state=0),
    param_distributions=param_dist,
    n_iter=HYPER_SEARCH_ITERS,
    cv=CV_FOLDS,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    random_state=0
)
search.fit(X, y)
best_model = search.best_estimator_
logger.info(f"Best params: {search.best_params_}")

# 11) Final evaluation on train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
logger.info(f"Test MAE: {mean_absolute_error(y_test, y_pred):.2f}")
logger.info(f"Test R2:  {r2_score(y_test, y_pred):.2f}")

# 12) Persist model
joblib.dump(best_model, MODEL_PATH)
logger.info(f"Saved model to {MODEL_PATH}")

# 13) Plot feature importances (matplotlib)
import matplotlib.pyplot as plt
importances = pd.Series(best_model.feature_importances_, index=X.columns).sort_values(ascending=False)
plt.figure(figsize=(8,6))
importances.plot(kind='barh')
plt.title('Feature Importances')
plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'farms_metadata.csv'

In [1]:

# Colab Cell: Synthetic Farm Metadata + Weather → Irrigation Model


# 1) Install dependencies (run once)
!pip install requests pandas scikit-learn tqdm tenacity joblib

import os
import time
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from tqdm.auto import tqdm
from tenacity import retry, stop_after_attempt, wait_exponential
import joblib

# 2) Set your OpenWeatherMap API key
OWM_API_KEY = '13453137d1b56511802304e7024fb57f'
os.environ['13453137d1b56511802304e7024fb57f'] = OWM_API_KEY

# 3) Synthetic farm metadata generator
def load_farms(num_farms=50, seed=0):
    """Generate synthetic farm metadata for POC."""
    np.random.seed(seed)
    lats = np.random.uniform(-60, 60, num_farms)
    lons = np.random.uniform(-180, 180, num_farms)
    soil_types = ['sandy', 'loamy', 'clay', 'silt']
    crop_types = ['rice', 'maize', 'soy', 'wheat', 'lettuce']
    return [
        {
            'id': i+1,
            'lat': float(lats[i]),
            'lon': float(lons[i]),
            'soil_type': np.random.choice(soil_types),
            'crop_type': np.random.choice(crop_types)
        }
        for i in range(num_farms)
    ]

# 4) OpenWeather provider adapter with retry
class OpenWeatherProvider:
    BASE_URL = 'https://api.openweathermap.org/data/3.0/onecall'

    def __init__(self, api_key: str):
        self.api_key = api_key

    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
    def fetch(self, lat: float, lon: float) -> dict:
        params = {
            'lat': lat,
            'lon': lon,
            'appid': self.api_key,
            'units': 'metric',
            'exclude': 'minutely,alerts'
        }
        r = requests.get(self.BASE_URL, params=params, timeout=10)
        r.raise_for_status()
        return r.json()

# 5) Helper to derive season (Northern Hemisphere)
def season_from_date(date):
    m = date.month
    if m in (3,4,5):   return 'spring'
    if m in (6,7,8):   return 'summer'
    if m in (9,10,11): return 'autumn'
    return 'winter'

# 6) Build time‑series of weather for each farm (single snapshot for POC)
farms = load_farms(num_farms=50, seed=42)
provider = OpenWeatherProvider(api_key=OWM_API_KEY)
now = datetime.now()

records = []
for farm in tqdm(farms, desc="Farms"):
    try:
        data = provider.fetch(farm['lat'], farm['lon'])
        rec = {
            'farm_id':     farm['id'],
            'latitude':    farm['lat'],
            'longitude':   farm['lon'],
            'soil_type':   farm['soil_type'],
            'crop_type':   farm['crop_type'],
            'temperature': data['current']['temp'],
            'humidity':    data['current']['humidity'],
            'pressure':    data['current']['pressure'],
            'dew_point':   data['current']['dew_point'],
            'uvi':         data['current']['uvi'],
            'clouds':      data['current']['clouds'],
            'visibility':  data['current']['visibility'],
            'wind_speed':  data['current']['wind_speed'],
            'rainfall':    data.get('daily',[{}])[0].get('rain', 0.0),
            'season':      season_from_date(now)
        }
        records.append(rec)
    except Exception as e:
        logging.warning(f"Failed to fetch farm {farm['id']}: {e}")
    time.sleep(1)  # avoid rate limits

df = pd.DataFrame(records)

# 7) One‑hot encode categorical features
df_encoded = pd.get_dummies(
    df,
    columns=['soil_type','crop_type','season'],
    drop_first=True
)

# 8) Synthetic target (replace with real irrigation data later)
np.random.seed(42)
df_encoded['irrigation_need'] = (
    100
    - df_encoded['rainfall'] * 3
    + (df_encoded['temperature'] - 20) * 2
    + df_encoded['wind_speed'] * 1.5
    + np.random.normal(0, 5, size=len(df_encoded))
)

# 9) Prepare features & target
feat_cols = [c for c in df_encoded.columns if c not in ['farm_id','irrigation_need']]
X, y = df_encoded[feat_cols], df_encoded['irrigation_need']

# 10) Cross‑validation & hyperparameter tuning
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf = RandomForestRegressor(random_state=42)
search = RandomizedSearchCV(rf, param_dist, n_iter=5, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1, random_state=42)
search.fit(X, y)
best_model = search.best_estimator_

# 11) Evaluate on held‐out split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
best_model.fit(X_train, y_train)
preds = best_model.predict(X_test)

print(f"Best params: {search.best_params_}")
print(f"MAE on test: {mean_absolute_error(y_test, preds):.2f}")
print(f"R² on test:  {r2_score(y_test, preds):.2f}")

# 12) Save model & dataset
joblib.dump(best_model, 'rf_irrigation_model.pkl')
df_encoded.to_csv('df_encoded.csv', index=False)

# 13) Quick feature importance check
importances = pd.Series(best_model.feature_importances_, index=feat_cols).sort_values(ascending=False)
print("\nFeature importances:\n", importances)


Farms:   0%|          | 0/50 [00:00<?, ?it/s]

Best params: {'n_estimators': 50, 'min_samples_leaf': 1, 'max_depth': 5}
MAE on test: 11.94
R² on test:  0.90

Feature importances:
 rainfall           0.604560
temperature        0.240750
longitude          0.053030
dew_point          0.043343
pressure           0.019698
humidity           0.015353
latitude           0.007141
wind_speed         0.005864
uvi                0.002946
soil_type_silt     0.002610
soil_type_loamy    0.001336
clouds             0.001179
crop_type_maize    0.001024
crop_type_soy      0.000510
crop_type_wheat    0.000319
soil_type_sandy    0.000179
crop_type_rice     0.000100
visibility         0.000059
dtype: float64
